In [ ]:
import librosa
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

0 --> Angry
1 --> Calm
2 --> Disgust
3 --> Fearful
4 --> Happy
5 --> Neutral
6 --> Sad
7 --> Surprised

In [ ]:
# Paths
dataset_path = '/Users/hongtan/Downloads/archive/audio_speech_actors_01-24'
input_file = '/Users/hongtan/Downloads/inside_out_clip.mp4'
output_srt_file = "/Users/hongtan/Desktop/sentimentsub/website/sentsub/media/captions/audio.srt"
output_wav_file = "/Users/hongtan/Downloads/output.wav"
output_folder = "/Users/hongtan/Desktop/segmented-audio/"
insane_input_file = '/Users/hongtan/Desktop/sentimentsub/speech-to-text/audio-test-files/insane.wav'

In [ ]:
# Function to extract features from each audio file
def extract_features(file_path):
    audio, sampling_rate = librosa.load(file_path, sr=22050, duration=None)
#     mfccs = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=30)
#     features = np.mean(mfccs.T, axis=0)
#     features=np.mean(librosa.feature.melspectrogram(y=audio, sr=sampling_rate).T,axis=0)
    
    features=np.array([])
    
    stft=np.abs(librosa.stft(audio))
    
    mfccs=np.mean(librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=30).T, axis=0)

    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T,axis=0)

    mel=np.mean(librosa.feature.melspectrogram(y=audio, sr=sampling_rate).T,axis=0)
    
    features=np.hstack((mfccs, chroma, mel))
    
    return features

# Function to preprocess the data
def preprocess_data(dataset_path):
    X = []
    y = []
    for folder in os.listdir(dataset_path):
        for file in os.listdir(os.path.join(dataset_path, folder)):
            if file.endswith('.wav'):
                
                emotion = file.split('-')[2]
                if int(emotion) == 1:
                    label = 'Neutral'
                elif int(emotion) == 2:
                    label = 'Neutral' # Calm
                elif int(emotion) == 3:
                    label = 'Happy'
                elif int(emotion) == 4:
                    label = 'Sad'
                elif int(emotion) == 5:
                    label = 'Angry'
                elif int(emotion) == 6:
                    label = 'Fearful'
                elif int(emotion) == 7:
                    label = 'Disgust'
                elif int(emotion) == 8:
                    label = 'Surprised'
                else:
                    label = 'UNK'
                    
                file_path = os.path.join(dataset_path, folder, file)
                features = extract_features(file_path)
                X.append(features)
                y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Preprocess the data
X, y = preprocess_data(dataset_path)

# Print the shape of the feature matrix and the label array
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

In [ ]:
# One-Hot encode y
OHE = OneHotEncoder()

y = np.array(y).reshape(-1,1)
y = OHE.fit_transform(y).toarray()

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29, test_size=0.20, shuffle=True)

print('X_train: {}'.format(X_train.shape))
print('y_train: {}'.format(y_train.shape))
print('X_test: {}'.format(X_test.shape))
print('y_test: {}'.format(y_test.shape))

In [ ]:
# Normalize data
SS = StandardScaler()

X_train = SS.fit_transform(X_train)
X_test = SS.transform(X_test)

print('X_train: {}'.format(X_train.shape))
print('y_train: {}'.format(y_train.shape))
print('X_test: {}'.format(X_test.shape))
print('y_test: {}'.format(y_test.shape))

In [ ]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

print('X_train: {}'.format(X_train.shape))
print('X_test: {}'.format(X_test.shape))

In [ ]:
# Function to plot Training Accuracy vs Validation Accuracy
def TrainVal_plot(history):
    val_acc = history.history['val_accuracy']
    train_acc = history.history['accuracy']

    epochs = range(0, 100)
    plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
    plt.plot(epochs, train_acc, 'r', label='Training Accuracy')
    plt.title('Training vs. Validation')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# Define input shape
input_shape = (X_train.shape[1], 1)

# Create model
model = Sequential()

model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=input_shape ,use_bias=False))
model.add(BatchNormalization())
model.add(MaxPooling1D())

model.add(Conv1D(filters=16, kernel_size=3, activation='relu', use_bias=False))
model.add(BatchNormalization())
model.add(MaxPooling1D())

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', use_bias=False))
model.add(BatchNormalization())
model.add(MaxPooling1D())

model.add(Flatten())

model.add(Dense(units=128, activation='relu', use_bias=False))
model.add(BatchNormalization())

model.add(Dropout(0.5))

model.add(Dense(units=7, activation='softmax'))

# adam = Adam(learning_rate=0.01)
# Compile model with appropriate loss function, optimizer and metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test),)

In [ ]:
print("Accuracy of our model on Train data : " , round(model.evaluate(X_train,y_train)[1]*100,2) , "%")

In [ ]:
print("Accuracy of our model on test data : " , round(model.evaluate(X_test,y_test)[1]*100,2) , "%")

In [ ]:
TrainVal_plot(history)